
# 使用Gradio构建交互式AI应用

本实验将指导你使用 **Gradio** 创建交互式的人工智能应用，重点是构建一个具有流式响应功能的可定制聊天机器人。

**Gradio** 是一个Python前端库，它能让你用极少的代码，轻松地为你的AI模型或任意Python函数构建一个美观、易用的Web用户界面。

## 准备工作

在开始之前，请确保你已经安装了必要的软件包：

In [ ]:
# 安装所需软件包
%pip install gradio openai python-dotenv

## 1. 配置你的环境

In [ ]:
import os
import gradio as gr
from openai import OpenAI
from dotenv import load_dotenv

# 从.env文件加载环境变量
load_dotenv()

# 初始化OpenAI客户端
# 注意：为了安全，生产环境中应始终使用环境变量，而不是直接在代码中写入密钥。
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 验证API密钥是否加载成功
if os.environ.get("OPENAI_API_KEY"):
    print("OpenAI API 密钥已成功加载。")
else:
    print("错误：未找到OpenAI API密钥！请在.env文件中配置。")

## 2. 你的第一个Gradio界面

让我们从创建一个最简单的Gradio界面开始。

In [ ]:
# 我们定义一个名为 echo 的简单函数，它接收一个消息并原样返回。
def echo(message):
    return f"你输入了: {message}"

# 创建一个基础的Gradio界面。gr.Interface() 会为你的函数生成一个Web UI。
# 它会自动创建一个网页，包含输入框、输出框和提交按钮。
# - fn: 指定要绑定的函数 (这里是 echo)。
# - inputs: 定义输入组件的类型 (这里是 "textbox")。
# - outputs: 定义输出组件的类型 (这里也是 "textbox")。
# - title 和 description: 用于提供界面的标题和描述，增强用户体验。
demo = gr.Interface(
    fn=echo,
    inputs="textbox",
    outputs="textbox",
    title="回声机器人",
    description="一个简单地重复你输入内容的机器人",
    allow_flagging="never" # 禁用标记功能，使界面更简洁
)

# 启动界面。这会在本地启动一个Web服务器，你可以通过浏览器访问。
demo.launch()

## 3. 构建一个基础的聊天机器人

接下来，让我们利用OpenAI构建一个简单的聊天机器人。

In [ ]:
# 定义一个函数，用于从OpenAI获取响应。这个过程我们在之前的实验中已经熟悉了。
def get_response(message):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "你是一个乐于助人的助手。"},
            {"role": "user", "content": message}
        ]
    )
    return response.choices[0].message.content

# 我们创建与之前完全相同的Gradio界面，但这次将 `fn` 参数替换为 `get_response` 函数。
# 当用户在输入框中输入消息并点击提交时，Gradio会自动调用 `get_response` 函数，
# 并将返回的AI响应显示在输出区域。
chatbot = gr.Interface(
    fn=get_response,
    inputs=gr.Textbox(placeholder="问我任何问题..."),
    outputs="text",
    title="简单AI聊天机器人",
    description="一个由OpenAI驱动的AI助手",
    allow_flagging="never"
)

# 启动聊天机器人界面
chatbot.launch()

## 4. 创建一个流式响应的聊天机器人

In [ ]:
# 定义一个函数来从OpenAI流式获取响应
def stream_response(message):
    stream = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "你是一个乐于助人的助手。"},
            {"role": "user", "content": message}
        ],
        stream=True
    )
    
    response = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            response += chunk.choices[0].delta.content
            # `yield` 是一个Python关键字，它允许函数在不终止的情况下返回一个值。
            # 这意味着函数可以稍后被再次调用，并从它离开的地方继续执行。
            # 这对于流式响应非常有用，因为它允许我们在生成部分结果时就将其返回。
            yield response

# 创建一个流式聊天机器人界面
streaming_chatbot = gr.Interface(
    fn=stream_response,
    inputs="textbox",
    outputs="text",
    title="流式AI聊天机器人",
    description="一个能够实时流式输出响应的AI助手",
    allow_flagging="never"
)

# 启动流式聊天机器人
streaming_chatbot.launch()

## 5. 构建一个有记忆的聊天机器人

现在，让我们创建一个更高级的、能够记住对话历史的聊天机器人。同样，我们已经在之前的实验中探讨过其实现原理，现在我们只是将其集成到Gradio界面中。

In [ ]:
# 定义一个名为 chat 的函数，它接收用户消息和由Gradio管理的对话历史
def chat(message, history):
    # 用系统提示初始化消息列表
    messages = [{"role": "system", "content": "你是一个乐于助人的助手。"}]
    
    # 遍历Gradio传入的对话历史（`history`），它是一个由 [用户消息, 助手消息] 组成的列表
    for user_msg, assistant_msg in history:
        messages.append({"role": "user", "content": user_msg})
        if assistant_msg:
            messages.append({"role": "assistant", "content": assistant_msg})
    
    # 将当前的用户消息添加到对话中
    messages.append({"role": "user", "content": message})
    
    # 从OpenAI获取流式响应
    stream = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        stream=True
    )
    
    # 逐块返回响应
    response = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            response += chunk.choices[0].delta.content
            yield response

# 使用 `gr.ChatInterface`，这是一个为聊天机器人量身定制的高级组件，它会自动处理对话历史的格式。
memory_chatbot = gr.ChatInterface(
    fn=chat,
    title="有记忆的AI聊天机器人",
    description="一个能记住你对话历史的AI助手。",
    examples=["给我讲讲机器学习", "神经网络是如何工作的？"],
    allow_flagging="never"
)

memory_chatbot.launch()

## 6. 应用示例：餐厅菜单生成器

In [ ]:
# 定义一个生成餐厅菜单的函数
def generate_menu(restaurant_name, cuisine_type, special_requirements="无"):
    prompt = f"""
请为名为“{restaurant_name}”的餐厅创建一个菜单，该餐厅主打“{cuisine_type}”菜系。
特殊要求: {special_requirements}

菜单应包含:
- 3道开胃菜
- 4道主菜
- 2道甜点

对于每道菜，请提供菜名、简短描述和价格。

请使用Markdown格式化你的响应，包含合适的标题、样式和章节。
在菜单顶部添加一段关于餐厅的简短介绍。
"""
    
    response = client.chat.completions.create(
        model="gpt-4o", # 使用更强大的模型以获得更好的格式化和创意
        messages=[
            {"role": "system", "content": "你是一位专业的餐厅顾问，擅长创作精美且格式规范的菜单。"},
            {"role": "user", "content": prompt}
        ]
    )
    
    return response.choices[0].message.content

# 创建一个菜单生成器界面
menu_generator = gr.Interface(
    fn=generate_menu,
    inputs=[
        gr.Textbox(label="餐厅名称"),
        gr.Textbox(label="菜系类型 (例如, 意大利菜, 日本料理)"),
        gr.Textbox(label="特殊要求 (可选)", placeholder="例如, 提供素食选项")
    ],
    outputs=gr.Markdown(label="生成的菜单"), # 使用Markdown组件以正确显示格式
    title="餐厅菜单生成器",
    description="用AI创建一个专业的餐厅菜单",
    allow_flagging="never"
)

# 启动菜单生成器
menu_generator.launch()

## 7. 进一步学习的资源

- [Gradio 官方文档](https://www.gradio.app/docs/)
- [OpenAI API 官方文档](https://platform.openai.com/docs/api-reference)